In [22]:
import pandas as pd
from collections import Counter
from transformers import BertModel, BertTokenizer
import torch
import numpy as np
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import hdbscan
from torch.utils.data import DataLoader, Dataset, random_split
import torch.nn as nn
import torch.nn.functional as F
import torchsummary
import torch.optim as optim
import wandb 

# Load the Datasets

In [2]:
train = pd.read_pickle("train.pk")
val = pd.read_pickle("val.pk")
test = pd.read_pickle("test.pk")

In [3]:
len(train["BookingCode"].unique())

100

In [4]:
full_ds = pd.concat([train, val, test])

label_dict = {i:  booking_code for i, booking_code in enumerate(full_ds["BookingCode"].unique().tolist())}
booking_code_to_label = {v: k for k, v in label_dict.items()}

# PartNumber
part_number_dict = {i:  part_number for i, part_number in enumerate(full_ds["PartNumber"].unique().tolist())}
part_number_to_label = {v: k for k, v in part_number_dict.items()}

# hdbscanCluster
cluster_dict = {i:  cluster_number for i, cluster_number in enumerate(full_ds["hdbscanCluster"].unique().tolist())}
cluster_to_label = {v: k for k, v in cluster_dict.items()}

In [5]:
# Convert to pytorch datasets

class CustomDataset(Dataset):
    def __init__(self, dataframe: pd.DataFrame, preprocess_transforms=None):
        self.raw_data = dataframe
        self.preprocess_transforms = preprocess_transforms

    def __len__(self):
        return len(self.raw_data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        row = self.raw_data.iloc[idx]
        
        sample = {
            "text": row["Description"],
            "text_emb": row["descriptionEmbeddings"],
            "part_num": part_number_to_label[row["PartNumber"]],
            "price": [row["Price"]],
            "price_norm": row["NormPrice"],
            "cluster": cluster_to_label[row["hdbscanCluster"]],
            "label": booking_code_to_label[row["BookingCode"]]
        }

        if self.preprocess_transforms is not None:
            sample = self.preprocess_transforms(sample)

        return sample


In [6]:
train_ds = CustomDataset(train)
val_ds = CustomDataset(val)
test_ds = CustomDataset(test)

In [7]:
# Dataloaders 

train_loader = DataLoader(train_ds, batch_size=256,  shuffle=True)
val_loader = DataLoader(val_ds, batch_size=256,  shuffle=True)
test_loader = DataLoader(test_ds, batch_size=256,  shuffle=True)

In [8]:
for batch in train_loader:
    break

In [9]:
batch["label"].shape

torch.Size([256])

In [10]:
train.columns

Index(['ID', 'PartNumber', 'Description', 'Count', 'SumPrice', 'BookingCode',
       'DocumentId', 'descriptionEmbeddings', 'Price', 'NormPrice',
       'hdbscanCluster'],
      dtype='object')

# Building the Model

In [52]:
class classifierModel(nn.Module):
    def __init__(self, device):
        super(classifierModel, self).__init__()

        self.device = device
        
        self.n_part_number = len(part_number_dict)
        self.n_cluster = len(cluster_dict)
        
        self.part_number_emb = nn.Linear(in_features=len(part_number_dict), out_features=768, bias=False, device=self.device)
        self.price_adapter = nn.Linear(in_features=1, out_features=768, bias=False, device=self.device)
        self.cluster_emb = nn.Linear(in_features=len(cluster_dict), out_features=768, bias=False, device=self.device)

        self.norm1 = nn.BatchNorm1d(768, device=self.device)
        self.linear1 = nn.Linear(in_features=768, out_features=384, device=self.device)
        self.dropout1 = nn.Dropout(p=0.5)

        self.norm2 = nn.BatchNorm1d(384, device=self.device)
        self.linear2 = nn.Linear(in_features=384, out_features=192, device=self.device) 
        self.dropout2 = nn.Dropout(p=0.5)
        
        self.norm3 = nn.BatchNorm1d(192, device=self.device)
        self.linear3 = nn.Linear(in_features=192, out_features=len(label_dict), device=self.device) # Classification Head
        self.dropout3 = nn.Dropout(p=0.5)

        self.relu = F.leaky_relu 


    def forward(self, sample: dict):
        text_emb = torch.tensor(sample["text_emb"], dtype=torch.float32).to(self.device)
        part_num = torch.tensor(sample["part_num"]).to(self.device)
        price = torch.tensor(sample["price_norm"], dtype=torch.float32).to(self.device)
        price = torch.reshape(price, [price.shape[0], 1])
        cluster = torch.tensor(sample["cluster"]).to(self.device)

        part_emb = self.part_number_emb(F.one_hot(part_num, num_classes=self.n_part_number).float())
        price_emb = self.price_adapter(price)
        cluster_emb = self.cluster_emb(F.one_hot(cluster, num_classes=self.n_cluster).float())

        out = torch.sum(torch.stack([text_emb, part_emb, price_emb, cluster_emb], dim=0), dim=0)

        out = self.norm1(out)
        out = self.dropout1(out)
        out = self.linear1(out)
        out = self.relu(out)
        
        out = self.norm2(out)
        out = self.dropout2(out)
        out = self.linear2(out)
        out = self.relu(out)
        
        out = self.norm3(out)
        out = self.dropout3(out)
        out = self.linear3(out)
        

        return out
               
            
    

In [53]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

model = classifierModel(device=device)

In [54]:

def train_model(model, train_loader, val_loader, epochs=10, lr=1e-4, l1=0, l2=0, run_name=None):
    # Initialize Weights & Biases
    wandb.init(
        project="Buchungscodes",
        config={
            "epochs": epochs,
            "learning_rate": lr,
            "architecture": str(model),
            "optimizer": "Adam",
            "loss_function": "CrossEntropyLoss",
            "l1": l1,  # Added L1 regularization to config
            "l2": l2,  # Added L2 regularization to config
            "run_name": run_name  # Added run_name to config
        },
        reinit=True
    )
    config = wandb.config
    device = model.device

    model = torch.compile(model)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config.learning_rate, weight_decay=l2)  # L2 regularization

    wandb.watch(model, log="all", log_freq=10)

    best_val_loss = float('inf')
    best_val_acc = 0.0

    for epoch in range(1, epochs + 1):
        model.train()
        running_loss = 0.0
        for batch in train_loader:
            labels = batch["label"].to(device)
            optimizer.zero_grad()
            outputs = model(batch)
            loss = criterion(outputs, labels)
            
            # L1 regularization
            if l1 > 0:
                l1_loss = sum(p.abs().sum() for p in model.parameters())
                loss += l1 * l1_loss

            loss.backward()
            optimizer.step()
            running_loss += loss.item() * labels.size(0)
        
        epoch_loss = running_loss / len(train_loader.dataset)
        
        # Validation 
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for batch in val_loader:
                labels = batch["label"].to(device)               
                outputs = model(batch)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * labels.size(0)
                _, preds = torch.max(outputs, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        
        epoch_val_loss = val_loss / len(val_loader.dataset)
        val_acc = correct / total

        # Log to wandb
        wandb.log({
            "Epoch": epoch,
            "Training Loss": epoch_loss,
            "Validation Loss": epoch_val_loss,
            "Validation Accuracy": val_acc
        })

        print(f"Epoch {epoch}/{epochs} | "
              f"Train Loss: {epoch_loss:.4f} | "
              f"Val Loss: {epoch_val_loss:.4f} | "
              f"Val Acc: {val_acc:.4f}", end="\r")
        
        # Define filenames with run_name if provided
        run_suffix = f"_{run_name}" if run_name else ""

        if epoch_val_loss < best_val_loss:
            best_val_loss = epoch_val_loss
            torch.save(model.state_dict(), f"best_val_loss{run_suffix}.pth")
            print(f"\nSaved Best Val Loss Model at epoch {epoch}")
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), f"best_val_acc{run_suffix}.pth")
            print(f"--> Saved Best Val Acc Model at epoch {epoch}")

    wandb.finish()

In [55]:
train_model(model, train_loader, val_loader, epochs=200, lr=2e-3, l2=0.001, run_name="Ridge_001")

/mnt/Lukas/Übung/.venv/lib/python3.10/site-packages/torch/_dynamo/utils.py:2132: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return node.target(*args, **kwargs)
/mnt/Lukas/Übung/.venv/lib/python3.10/site-packages/torch/_dynamo/utils.py:2132: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return node.target(*args, **kwargs)
/mnt/Lukas/Übung/.venv/lib/python3.10/site-packages/torch/_dynamo/utils.py:2132: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return node.target(*args, **kwargs)
/mnt/Lukas/Übung/.venv/lib/python3.10/site-packages/tor

Epoch 1/200 | Train Loss: 4.0318 | Val Loss: 3.3407 | Val Acc: 0.3143
Saved Best Val Loss Model at epoch 1
--> Saved Best Val Acc Model at epoch 1
Epoch 2/200 | Train Loss: 2.8631 | Val Loss: 2.3402 | Val Acc: 0.4898
Saved Best Val Loss Model at epoch 2
--> Saved Best Val Acc Model at epoch 2
Epoch 3/200 | Train Loss: 2.2949 | Val Loss: 1.9065 | Val Acc: 0.5592
Saved Best Val Loss Model at epoch 3
--> Saved Best Val Acc Model at epoch 3
Epoch 4/200 | Train Loss: 2.0058 | Val Loss: 1.6557 | Val Acc: 0.5946
Saved Best Val Loss Model at epoch 4
--> Saved Best Val Acc Model at epoch 4
Epoch 5/200 | Train Loss: 1.7807 | Val Loss: 1.4769 | Val Acc: 0.6190
Saved Best Val Loss Model at epoch 5
--> Saved Best Val Acc Model at epoch 5
Epoch 6/200 | Train Loss: 1.6253 | Val Loss: 1.3856 | Val Acc: 0.6435
Saved Best Val Loss Model at epoch 6
--> Saved Best Val Acc Model at epoch 6
Epoch 7/200 | Train Loss: 1.5006 | Val Loss: 1.2780 | Val Acc: 0.6694
Saved Best Val Loss Model at epoch 7
--> Saved B

Epoch,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
Training Loss,█▆▆▄▄▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation Accuracy,▁▃▅▆▆▇▇▇▇▇█▇▇▇▇██████████▇██████████████
Validation Loss,█▇▆▅▅▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▂▁▁▂▂▂
Epoch,200
Training Loss,0.30743
Validation Accuracy,0.82993
Validation Loss,0.6617


# Feature Selection 

In [41]:
class justTextModel(nn.Module):
    def __init__(self, device):
        super(justTextModel, self).__init__()

        self.device = device
        
        self.norm1 = nn.BatchNorm1d(768, device=self.device)
        self.linear1 = nn.Linear(in_features=768, out_features=384, device=self.device)
        self.dropout1 = nn.Dropout(p=0.5)

        self.norm2 = nn.BatchNorm1d(384, device=self.device)
        self.linear2 = nn.Linear(in_features=384, out_features=192, device=self.device) 
        self.dropout2 = nn.Dropout(p=0.5)
        
        self.norm3 = nn.BatchNorm1d(192, device=self.device)
        self.linear3 = nn.Linear(in_features=192, out_features=len(label_dict), device=self.device) # Classification Head
        self.dropout3 = nn.Dropout(p=0.5)

        self.relu = F.leaky_relu 


    def forward(self, sample: dict):
        text_emb = torch.tensor(sample["text_emb"], dtype=torch.float32).to(self.device)

        out = self.norm1(text_emb)
        out = self.dropout1(out)
        out = self.linear1(out)
        out = self.relu(out)
        
        out = self.norm2(out)
        out = self.dropout2(out)
        out = self.linear2(out)
        out = self.relu(out)
        
        out = self.norm3(out)
        out = self.dropout3(out)
        out = self.linear3(out)
        

        return out

In [42]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

just_text_model = classifierModel(device=device)

train_model(model, train_loader, val_loader, epochs=100, lr=2e-3, l2=0.001, run_name="Just_Text_Base")

Epoch 1/100 | Train Loss: 0.1778 | Val Loss: 0.8888 | Val Acc: 0.8204
Saved Best Val Loss Model at epoch 1
--> Saved Best Val Acc Model at epoch 1
Epoch 3/100 | Train Loss: 0.1161 | Val Loss: 0.8700 | Val Acc: 0.8204
Saved Best Val Loss Model at epoch 3
--> Saved Best Val Acc Model at epoch 4Loss: 0.9041 | Val Acc: 0.8259
Epoch 7/100 | Train Loss: 0.1258 | Val Loss: 0.8654 | Val Acc: 0.8218
Saved Best Val Loss Model at epoch 7
Epoch 8/100 | Train Loss: 0.1334 | Val Loss: 0.8652 | Val Acc: 0.8204
Saved Best Val Loss Model at epoch 8
Epoch 24/100 | Train Loss: 0.1137 | Val Loss: 0.8643 | Val Acc: 0.8231
Saved Best Val Loss Model at epoch 24
Epoch 25/100 | Train Loss: 0.1015 | Val Loss: 0.8620 | Val Acc: 0.8190
Saved Best Val Loss Model at epoch 25
--> Saved Best Val Acc Model at epoch 26Loss: 0.8626 | Val Acc: 0.8272
Epoch 32/100 | Train Loss: 0.1479 | Val Loss: 0.8602 | Val Acc: 0.8272
Saved Best Val Loss Model at epoch 32
Epoch 34/100 | Train Loss: 0.1227 | Val Loss: 0.8375 | Val Acc: 

Epoch,▁▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇█
Training Loss,█▅▃▄▁▃▄▃▄▃▂▃▅▅▃█▅▃▂▃▂▁▄▄▆▂▂▆▂▁▆█▆█▇▃▂▄▄▂
Validation Accuracy,▇▃▆▇▇▅▇▆▆▇▇▅▆▅▄▆█▅▃▇▅▅▅▅▄▃▄▅▅▅▃▄▁▄▆▅▂▄▅▅
Validation Loss,▄▄▄▄▂▃▃▄▄▄▄▄▃▂▄▁▂▃▂▂▂▃▂▃▃▁▂▁▁▃█▃▂▂▅▄▄▂▃▄
Epoch,100
Training Loss,0.10718
Validation Accuracy,0.81361
Validation Loss,0.87889
